In [11]:
import numpy as np
from random import choices, randrange
import random
import matplotlib.pyplot as plt
from numba import jit

from utility import *

In [12]:
@jit(nopython=True)
def change_sol(sol, k):
    new_sol = sol.copy()
    n = randrange(len(new_sol))
    old_value = new_sol[n]

    while old_value == new_sol[n]:
        new_sol[n] = randrange(K)

    return new_sol

@jit(nopython=True)
def simulated_annealing(iters, alpha, base_sol, points, K, evaluate, make_a_move):
    best_sol = base_sol
    base_val = evaluate(best_sol, points, K)
    best_value = base_val
    T = base_val/500
    Tf = 10**-3
    curr_sol = base_sol
    curr_val = best_value
    iter = 1

    old_sol = base_sol
    finito = False
    no_update = 0
    while(finito == False):
        print("Iteration number:", iter, "Best value percentuale: ", curr_val/base_val*100, "% T:", T)
        iter = 1 + iter
        old_sol = curr_sol
        
        for i in range(iters):
            candidate = make_a_move(curr_sol, K)
            val_candidate = evaluate(candidate, points, K)

            if(val_candidate < best_value):
                best_value = val_candidate
                best_sol = candidate
            
            if(val_candidate < curr_val):
                curr_val = val_candidate
                curr_sol = candidate
                no_update = 0
            else:
                r = random.uniform(0, 1)
                delta = abs(curr_val - val_candidate)
                tresh = np.exp(-delta/T)
                no_update = 0
                if(r < tresh):
                    curr_val = val_candidate
                    curr_sol = candidate
                else:
                    no_update = no_update + 1

        T = alpha*T
    
        if(no_update == 3000 or T <= Tf):
            finito = True
    return best_sol

In [13]:
points = load_points('C:/Users/franc/Desktop/prova.txt')

K = 15
N = len(points)

sol = np.random.randint(K,size = N)
#sol = create_initial_sol(points, K)
sol = simulated_annealing(500, 0.95, sol, points, K, squared_inner_distance, change_sol)

print("{:.5E}".format(squared_inner_distance(sol, points, K)))

Iteration number: 1 Best value percentuale:  100.0 % T: 1150649625903.3044
Iteration number: 2 Best value percentuale:  99.9971973065086 % T: 1144896377773.7878
Iteration number: 3 Best value percentuale:  100.00266826396897 % T: 1139171895884.919
Iteration number: 4 Best value percentuale:  100.00339902802489 % T: 1133476036405.4944
Iteration number: 5 Best value percentuale:  100.00624150603754 % T: 1127808656223.4668
Iteration number: 6 Best value percentuale:  100.01028979427491 % T: 1122169612942.3494
Iteration number: 7 Best value percentuale:  100.01237790793111 % T: 1116558764877.6377
Iteration number: 8 Best value percentuale:  100.00436892237028 % T: 1110975971053.2495
Iteration number: 9 Best value percentuale:  100.00302383659397 % T: 1105421091197.9832
Iteration number: 10 Best value percentuale:  99.99922261989268 % T: 1099893985741.9932
Iteration number: 11 Best value percentuale:  99.99971092872352 % T: 1094394515813.2832
Iteration number: 12 Best value percentuale:  99